In [2]:
from pathlib import Path
import sys


here = Path.cwd()
candidates = [here] + list(here.parents)
for p in candidates:
    if (p / "Code").is_dir():
        sys.path.insert(0, str(p))
        break
else:
    raise RuntimeError("Couldn't find a 'Code' folder in this project.")


import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd


from Code.PINN import create_network_model, u, compute_loss, g_trial_tf, make_train_step, compute_MSE
from Code.functions import euler, analytical_u

In [3]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
N_points = 1000  # Number of collocation points (x, t) in the domain
epochs = 3000  # Number of training epochs

T_final = 0.5 # Example final time
x_samples = np.random.uniform(0.0, 1.0, N_points)
t_samples = np.random.uniform(0.0, T_final, N_points)
X_train = np.stack([x_samples, t_samples], axis=1).astype(np.float32)


2025-12-12 11:59:13.555610: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-12-12 11:59:13.555646: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-12-12 11:59:13.555652: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-12-12 11:59:13.555688: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-12-12 11:59:13.555702: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:

widths = [5, 10, 20, 50, 100]
results_nodes = []

epochs = 3000 

for w in widths:
    print(f"\n=== Width: {w} ===")
    
    model = create_network_model(layers=[w], activation='tanh')
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

    train_step = make_train_step(model, optimizer, compute_loss)

    for epoch in range(epochs):
        loss = train_step(X_train)
        if epoch % 500 == 0:
            print(f"  Epoch {epoch}: loss = {loss.numpy():.3e}")

    mse = compute_MSE(model, T_final=T_final)
    results_nodes.append((w, float(loss.numpy()), mse))
    print(f"Final residual loss: {loss.numpy():.3e}, MSE vs exact: {mse:.3e}")

df_nodes = pd.DataFrame(results_nodes, columns=["Width", "Residual loss", "MSE"])
print(df_nodes)



=== Width: 5 ===


2025-12-12 11:59:14.133963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


  Epoch 0: loss = 2.017e+01
  Epoch 500: loss = 7.919e+00
  Epoch 1000: loss = 7.672e+00
  Epoch 1500: loss = 1.559e+00
  Epoch 2000: loss = 1.118e+00
  Epoch 2500: loss = 8.975e-01
Final residual loss: 7.639e-01, MSE vs exact: 1.030e-03

=== Width: 10 ===
  Epoch 0: loss = 2.127e+01
  Epoch 500: loss = 1.051e+00
  Epoch 1000: loss = 2.671e-01
  Epoch 1500: loss = 1.205e-01
  Epoch 2000: loss = 7.406e-02
  Epoch 2500: loss = 5.223e-02
Final residual loss: 3.752e-02, MSE vs exact: 1.893e-05

=== Width: 20 ===
  Epoch 0: loss = 2.307e+01
  Epoch 500: loss = 8.676e-01
  Epoch 1000: loss = 1.745e-01
  Epoch 1500: loss = 5.341e-02
  Epoch 2000: loss = 2.510e-02
  Epoch 2500: loss = 1.379e-02
Final residual loss: 9.368e-03, MSE vs exact: 2.246e-06

=== Width: 50 ===
  Epoch 0: loss = 2.298e+01
  Epoch 500: loss = 5.327e-01
  Epoch 1000: loss = 4.198e-02
  Epoch 1500: loss = 1.792e-02
  Epoch 2000: loss = 9.757e-03
  Epoch 2500: loss = 4.999e-03
Final residual loss: 2.851e-03, MSE vs exact: 1

In [5]:
depth_architectures = [
    [50],
    [50, 50],
    [50, 50, 50],
    [50, 50, 50, 50],
]

results_layers = []

for layers in depth_architectures:
    print(f"\n=== Depth experiment: layers={layers}, tanh ===")
    # Build new model
    model = create_network_model(layers=layers)
    
    # IMPORTANT: new optimizer for each model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    
    # Reset traced graph: DON'T reuse old tf.function trace
    train_step = make_train_step(model, optimizer, compute_loss)

    # Train
    for epoch in range(epochs):
        loss = train_step(X_train)
        if epoch % 500 == 0:
            print(f"  Epoch {epoch}: loss = {loss.numpy():.3e}")

    mse = compute_MSE(model, T_final=T_final)
    results_layers.append((str(layers), float(loss.numpy()), mse))
    print(f"Final residual loss: {loss.numpy():.3e}, MSE vs exact: {mse:.3e}")

df_layers = pd.DataFrame(results_layers, columns=["Architecture", "Residual loss", "MSE"])
print(df_layers)


=== Depth experiment: layers=[50], tanh ===
  Epoch 0: loss = 2.046e+01
  Epoch 500: loss = 6.492e-01
  Epoch 1000: loss = 4.560e-02
  Epoch 1500: loss = 1.680e-02
  Epoch 2000: loss = 9.169e-03
  Epoch 2500: loss = 5.333e-03
Final residual loss: 4.394e-03, MSE vs exact: 7.922e-06

=== Depth experiment: layers=[50, 50], tanh ===
  Epoch 0: loss = 2.015e+01
  Epoch 500: loss = 6.615e-03
  Epoch 1000: loss = 3.569e-02
  Epoch 1500: loss = 5.640e-02
  Epoch 2000: loss = 2.394e-04
  Epoch 2500: loss = 3.857e-04
Final residual loss: 1.826e-04, MSE vs exact: 7.792e-08

=== Depth experiment: layers=[50, 50, 50], tanh ===
  Epoch 0: loss = 2.305e+01
  Epoch 500: loss = 4.001e-03
  Epoch 1000: loss = 7.092e-04
  Epoch 1500: loss = 3.173e-04
  Epoch 2000: loss = 3.203e-04
  Epoch 2500: loss = 9.808e-03
Final residual loss: 1.263e-02, MSE vs exact: 6.995e-05

=== Depth experiment: layers=[50, 50, 50, 50], tanh ===
  Epoch 0: loss = 2.213e+01
  Epoch 500: loss = 6.625e-03
  Epoch 1000: loss = 8.9

In [6]:
activations = ['tanh', 'sigmoid', 'relu', 'swish']

results_activations = []
epochs = 3000

for act in activations:
    print(f"\n=== Activation experiment: {act}, layers=[50,50] ===")

    # 1. Create model with the activation
    model = create_network_model(layers=[50, 50], activation=act)

    # 2. New optimizer per model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

    # 3. Create a NEW train_step bound to this model & optimizer
    train_step = make_train_step(model, optimizer, compute_loss)

    # 4. Training loop
    for epoch in range(epochs):
        loss = train_step(X_train)
        if epoch % 500 == 0:
            print(f"  Epoch {epoch}: loss = {loss.numpy():.3e}")

    # 5. Compute accuracy
    mse = compute_MSE(model, T_final=T_final)

    results_activations.append((act, float(loss.numpy()), mse))

    print(f"Final residual loss: {loss.numpy():.3e}, MSE vs exact: {mse:.3e}")

df_act = pd.DataFrame(results_activations, columns=["Activation", "Residual loss", "MSE"])
print(df_act)



=== Activation experiment: tanh, layers=[50,50] ===
  Epoch 0: loss = 2.220e+01
  Epoch 500: loss = 6.032e-03
  Epoch 1000: loss = 9.157e-04
  Epoch 1500: loss = 4.217e-04
  Epoch 2000: loss = 2.431e-03
  Epoch 2500: loss = 1.421e-03
Final residual loss: 1.727e-04, MSE vs exact: 5.609e-08

=== Activation experiment: sigmoid, layers=[50,50] ===
  Epoch 0: loss = 1.738e+01
  Epoch 500: loss = 7.475e-02
  Epoch 1000: loss = 6.203e-03
  Epoch 1500: loss = 1.798e-03
  Epoch 2000: loss = 7.606e-04
  Epoch 2500: loss = 4.049e-04
Final residual loss: 2.542e-04, MSE vs exact: 3.463e-08

=== Activation experiment: relu, layers=[50,50] ===
  Epoch 0: loss = 2.031e+01
  Epoch 500: loss = 1.220e+00
  Epoch 1000: loss = 3.752e+00
  Epoch 1500: loss = 2.063e+00
  Epoch 2000: loss = 2.335e+00
  Epoch 2500: loss = 2.392e+00
Final residual loss: 2.308e+00, MSE vs exact: 1.037e+01

=== Activation experiment: swish, layers=[50,50] ===
  Epoch 0: loss = 2.119e+01
  Epoch 500: loss = 3.703e-03
  Epoch 1000

In [7]:
eta = [0.0001, 0.001, 0.01, 0.1, 1.0]
epochs = 5000

results_eta = []

for learning_rate in eta:
    print(f"\n=== Learning rate experiment: eta={learning_rate}, layers=[50,50], swish ===")

    # 1. Create model
    model = create_network_model(layers=[50, 50], activation='swish')

    # 2. New optimizer per model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # 3. Create a NEW train_step bound to this model & optimizer
    train_step = make_train_step(model, optimizer, compute_loss)

    # 4. Training loop
    for epoch in range(epochs):
        loss = train_step(X_train)
        if epoch % 500 == 0:
            print(f"  Epoch {epoch}: loss = {loss.numpy():.3e}")

    # 5. Compute accuracy
    mse = compute_MSE(model, T_final=T_final)

    results_eta.append((learning_rate, float(loss.numpy()), mse))

    print(f"Final residual loss: {loss.numpy():.3e}, MSE vs exact: {mse:.3e}")
    
df_eta = pd.DataFrame(results_eta, columns=["Learning rate", "Residual loss", "MSE"])
print(df_eta)


=== Learning rate experiment: eta=0.0001, layers=[50,50], swish ===
  Epoch 0: loss = 2.124e+01
  Epoch 500: loss = 1.401e+01
  Epoch 1000: loss = 1.050e+01
  Epoch 1500: loss = 9.596e+00
  Epoch 2000: loss = 8.882e+00
  Epoch 2500: loss = 7.953e+00
  Epoch 3000: loss = 6.453e+00
  Epoch 3500: loss = 4.280e+00
  Epoch 4000: loss = 2.212e+00
  Epoch 4500: loss = 1.244e+00
Final residual loss: 1.047e+00, MSE vs exact: 2.189e-03

=== Learning rate experiment: eta=0.001, layers=[50,50], swish ===
  Epoch 0: loss = 2.132e+01
  Epoch 500: loss = 1.129e+00
  Epoch 1000: loss = 1.074e-01
  Epoch 1500: loss = 2.042e-02
  Epoch 2000: loss = 8.716e-03
  Epoch 2500: loss = 5.091e-03
  Epoch 3000: loss = 3.108e-03
  Epoch 3500: loss = 1.719e-03
  Epoch 4000: loss = 8.770e-04
  Epoch 4500: loss = 4.837e-04
Final residual loss: 3.095e-04, MSE vs exact: 4.653e-08

=== Learning rate experiment: eta=0.01, layers=[50,50], swish ===
  Epoch 0: loss = 2.149e+01
  Epoch 500: loss = 3.847e-03
  Epoch 1000: 

In [8]:
import random

def set_seed(seed):
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


seeds = [0, 1, 2, 3, 4]
epochs = 5000

results_seeds = []

for seed in seeds:
    print(f"\n=== Seed experiment: seed={seed}, layers=[50,50], swish ===")

    # --- 1. Set seed BEFORE building the model ---
    set_seed(seed)

    # --- 2. Create model ---
    model = create_network_model(layers=[50, 50], activation='swish')

    # --- 3. Optimizer ---
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # --- 4. New train_step for this model ---
    train_step = make_train_step(model, optimizer, compute_loss)

    # --- 5. Training loop ---
    for epoch in range(epochs):
        loss = train_step(X_train)
        if epoch % 500 == 0:
            print(f"  Epoch {epoch}: loss = {loss.numpy():.3e}")

    # --- 6. Compute accuracy ---
    mse = compute_MSE(model, T_final=T_final)

    results_seeds.append((seed, float(loss.numpy()), mse))

    print(f"Final residual loss: {loss.numpy():.3e}, MSE vs exact: {mse:.3e}")

df_seeds = pd.DataFrame(results_seeds, columns=["Seed", "Residual loss", "MSE"])
print(df_seeds)




=== Seed experiment: seed=0, layers=[50,50], swish ===
  Epoch 0: loss = 2.102e+01
  Epoch 500: loss = 1.139e+00
  Epoch 1000: loss = 1.578e-01
  Epoch 1500: loss = 3.358e-02
  Epoch 2000: loss = 1.353e-02
  Epoch 2500: loss = 6.635e-03
  Epoch 3000: loss = 4.002e-03
  Epoch 3500: loss = 2.418e-03
  Epoch 4000: loss = 1.478e-03
  Epoch 4500: loss = 9.980e-04
Final residual loss: 7.295e-04, MSE vs exact: 1.365e-07

=== Seed experiment: seed=1, layers=[50,50], swish ===
  Epoch 0: loss = 2.120e+01
  Epoch 500: loss = 1.255e+00
  Epoch 1000: loss = 4.161e-01
  Epoch 1500: loss = 3.060e-02
  Epoch 2000: loss = 1.097e-02
  Epoch 2500: loss = 5.415e-03
  Epoch 3000: loss = 2.738e-03
  Epoch 3500: loss = 1.391e-03
  Epoch 4000: loss = 7.983e-04
  Epoch 4500: loss = 5.161e-04
Final residual loss: 3.504e-04, MSE vs exact: 5.387e-08

=== Seed experiment: seed=2, layers=[50,50], swish ===
  Epoch 0: loss = 2.105e+01
  Epoch 500: loss = 1.125e+00
  Epoch 1000: loss = 2.493e-01
  Epoch 1500: loss 